In [5]:
#Python 2.x program for Speech Recognition

import speech_recognition as sr

#enter the name of usb microphone that you found
#using lsusb
#the following name is only used as an example
mic_name = "USB Device 0x46d:0x825: Audio (hw:1, 0)"
#Sample rate is how often values are recorded
sample_rate = 48000
#Chunk is like a buffer. It stores 2048 samples (bytes of data)
#here.
#it is advisable to use powers of 2 such as 1024 or 2048
chunk_size = 2048
#Initialize the recognizer
r = sr.Recognizer()

#generate a list of all audio cards/microphones
mic_list = sr.Microphone.list_microphone_names()
device_id=0

#the following loop aims to set the device ID of the mic that
#we specifically want to use to avoid ambiguity.
for i, microphone_name in enumerate(mic_list):
	if microphone_name == mic_name:
		device_id = i

#use the microphone as source for input. Here, we also specify
#which device ID to specifically look for incase the microphone
#is not working, an error will pop up saying "device_id undefined"
with sr.Microphone(device_index = device_id, sample_rate = sample_rate,
						chunk_size = chunk_size) as source:
	#wait for a second to let the recognizer adjust the
	#energy threshold based on the surrounding noise level
	r.adjust_for_ambient_noise(source)
	print ("Say Something")
	#listens for the user's input
	audio = r.listen(source)
		
	try:
		text = r.recognize_google(audio)
		print ("you said: " + text)
	
	#error occurs when google could not understand what was said
	
	except sr.UnknownValueError:
		print("Google Speech Recognition could not understand audio")
	
	except sr.RequestError as e:
		print("Could not request results from Google speech Recognition service; {0}".format(e))


Say Something


OSError: FLAC conversion utility not available - consider installing the FLAC command line application by running `apt-get install flac` or your operating system's equivalent

In [1]:
import pyaudio as pa
help(pa)

Help on module pyaudio:

NAME
    pyaudio

DESCRIPTION
    PyAudio provides Python bindings for PortAudio, the cross-platform
    audio I/O library. With PyAudio, you can easily use Python to play and
    record audio on a variety of platforms.  PyAudio is inspired by:
    
    * pyPortAudio/fastaudio: Python bindings for PortAudio v18 API.
    * tkSnack: cross-platform sound toolkit for Tcl/Tk and Python.
    
    .. include:: ../sphinx/examples.rst
    
    Overview
    --------
    
    **Classes**
      :py:class:`PyAudio`, :py:class:`Stream`
    
    .. only:: pamac
    
       **Host Specific Classes**
         :py:class:`PaMacCoreStreamInfo`
    
    **Stream Conversion Convenience Functions**
      :py:func:`get_sample_size`, :py:func:`get_format_from_width`
    
    **PortAudio version**
      :py:func:`get_portaudio_version`, :py:func:`get_portaudio_version_text`
    
    .. |PaSampleFormat| replace:: :ref:`PortAudio Sample Format <PaSampleFormat>`
    .. _PaSampleFormat:
   

In [17]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [1]:
import pyaudio
import wave
import sys

CHUNK = 1024

if len(sys.argv) < 2:
    print("Plays a wave file. output.wav")
    sys.exit(-1)

wf = wave.open('output.wav', 'rb')

# instantiate PyAudio (1)
p = pyaudio.PyAudio()

# open stream (2)
stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

# read data
data = wf.readframes(CHUNK)

# play stream (3)
while len(data) > 0:
    stream.write(data)
    data = wf.readframes(CHUNK)

# stop stream (4)
stream.stop_stream()
stream.close()

# close PyAudio (5)
p.terminate()